# Implementation of PCA from scratch
### Source: https://bagheri365.github.io/blog/Principal-Component-Analysis-from-Scratch/
### eigenvectors and eigenvalues: https://medium.com/swlh/eigen-theory-from-the-scratch-a73e0b5a25da

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']

n_samples, n_features = X.shape

print('Number of samples:', n_samples)
print('Number of features:', n_features)

Number of samples: 150
Number of features: 4


In [3]:
data = pd.DataFrame(X, columns=iris.feature_names)
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


# Paso 1: Estandarizar datos

In [4]:
def scaler(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [5]:
X_std = scaler(X)

# Find the Covariance matrix of the standarized data

In [6]:
#correlation_matrix = np.corrcoef(X_std.T)
#correlation_matrix
cov_matrix = np.cov(X_std.T)
cov_matrix

array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343066],
       [-0.11835884,  1.00671141, -0.43131554, -0.36858315],
       [ 0.87760447, -0.43131554,  1.00671141,  0.96932762],
       [ 0.82343066, -0.36858315,  0.96932762,  1.00671141]])

# Step 3: Find the eigenvectors and eigenvalues of the covariance matrix (Valores propios y vectores propios)

In [7]:
eig_vals, eig_vecs = np.linalg.eig(cov_matrix)
eig_vecs = eig_vecs.T
print('Eigenvalues: \n', eig_vals)
print('Eigenvectors: \n', eig_vecs)

Eigenvalues: 
 [2.93808505 0.9201649  0.14774182 0.02085386]
Eigenvectors: 
 [[ 0.52106591 -0.26934744  0.5804131   0.56485654]
 [-0.37741762 -0.92329566 -0.02449161 -0.06694199]
 [-0.71956635  0.24438178  0.14212637  0.63427274]
 [ 0.26128628 -0.12350962 -0.80144925  0.52359713]]


In [8]:

#max_abs_idx = np.argmax(np.abs(eig_vecs), axis=0)
#signs = np.sign(eig_vecs[max_abs_idx, range(eig_vecs.shape[0])])
#eig_vecs = eig_vecs*signs[np.newaxis,:]
#eig_vecs = eig_vecs.T
#max_abs_idx
#eig_vecs

# Matriz de transformación para k dado

In [9]:
# Order eigenvectors based on eigenvalues
paired_eig = [(eig_vals[i], eig_vecs[i, :]) for i in range(len(eig_vals))]
paired_eig.sort(key=lambda x: x[0], reverse=True)
sorted_vals = np.array([i[0] for i in paired_eig])
sorted_vecs = np.array([i[1] for i in paired_eig])
paired_eig

[(np.float64(2.9380850501999927),
  array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])),
 (np.float64(0.920164904162486),
  array([-0.37741762, -0.92329566, -0.02449161, -0.06694199])),
 (np.float64(0.14774182104494718),
  array([-0.71956635,  0.24438178,  0.14212637,  0.63427274])),
 (np.float64(0.02085386217646255),
  array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))]

In [10]:
k = 2

In [11]:
# Get fitst k vectors based on eigenvalues

w = sorted_vecs[:k, :]
w

array([[ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654],
       [-0.37741762, -0.92329566, -0.02449161, -0.06694199]])

# Transformar las variables originales estandarizadas con la matriz de transformación

In [12]:
result = (w @ X_std.T).T 

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=k)
fit = pca.fit(X_std)
#fit.explained_variance_
pca.components_
#transformed_data = fit.transform(X_std)
#transformed_data

array([[ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654],
       [ 0.37741762,  0.92329566,  0.02449161,  0.06694199]])

In [14]:

def PCA_fit_transform(self, X, n_components):
    # Estandarizar datos
    X = X.copy()
    mean = np.mean(X, axis = 0)
    scale = np.std(X, axis = 0)
    X_std = (X - mean) / scale

    # Matriz de correlacion (covarianzas estandarizadas)    
    cov_mat = np.cov(X_std.T)

    # Valores y vectores propios
    eig_vals, eig_vecs = np.linalg.eig(cov_mat) 
    eig_vecs = eig_vecs.T

   # Ordenar vectores propios en orden descendente (matriz transformación)
    paired_eig = [(eig_vals[i], eig_vecs[i, :]) for i in range(len(eig_vals))]
    paired_eig.sort(key=lambda x: x[0], reverse=True)
    sorted_vals = np.array([i[0] for i in paired_eig])
    sorted_vecs = np.array([i[1] for i in paired_eig])
    
    # Matriz de transformación
    w = sorted_vecs[:n_components, :]

    # Cálculo de componentes
    result = (w @ X_std.T).T 
    
    # Componentes
    return result